In [23]:
import pandas as pd
import os
import numpy as np
import wandb

In [ ]:
!wandb login

In [ ]:
from google.colab import drive
import os

# 1. Mounting Google Drive: This allows Colab to access files in your Google Drive
drive.mount('/content/drive')
basedir = "/content/drive/MyDrive/cs6365/"

In [25]:
df21 = pd.read_csv('/content/drive/MyDrive/cs6365/merged_gw_2021_22.csv')
df22 = pd.read_csv('/content/drive/MyDrive/cs6365/merged_gw_2022_23.csv')
df23 = pd.read_csv('/content/drive/MyDrive/cs6365/merged_gw_2023_24.csv')

In [ ]:
df23.columns

Columns to drop: Name, team, position, element, fixture, kickoff time (tbd), opponent team, round, transfer_balance


Target data: total_points of next gameweek

preprocessing steps:
- Change was_home to 0/1

# Quick and dirty baseline model

In [ ]:
# Dropping unnecessary columns
train_df = df21.drop(["name", "team", "position", "fixture", "kickoff_time", "opponent_team", "round", "transfers_balance"], axis=1)

# Create a column for ground truth
ground_truth = df21.loc[:, ["element", "total_points", "GW"]]

# total points for GW 2 becomes the ground truth for GW 1
ground_truth['GW'] -= 1
ground_truth = ground_truth.rename(columns={"total_points":"y"})

# inner join with train df
train_df = train_df.merge(ground_truth, how="inner", on=["element", "GW"])

# Dropping 'element' and 'gw' columns as they are no longer needed
train_df = train_df.drop(["element", "GW"], axis=1)
train_df

In [ ]:
#Pre-processing
train_df = train_df.astype(np.float32)
train_df

In [7]:
# Train model
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [8]:
class FplDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return torch.tensor(self.inputs[idx], dtype=torch.float32), torch.tensor(self.targets[idx], dtype=torch.float32)

# Convert data to PyTorch Dataset and DataLoader
# inputs = train_df.drop('y', axis=1)
# num_features = inputs.shape[1]
# targets = train_df.loc[:,"y"]
# dataset = FplDataset(inputs, targets)
# dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
# Define a simple model
class SimpleModel(nn.Module):
    def __init__(self, num_features):
        super(SimpleModel, self).__init__()
        self.layer1 = nn.Linear(num_features, 64)  # 64 output units
        self.layer2 = nn.Linear(64, 1)   # 64 input units, 1 output value

    def forward(self, x):
        x = torch.relu(self.layer1(x))  # Apply ReLU activation after first layer
        x = self.layer2(x)              # Output layer
        return x

# Initialize the model, loss function, and optimizer
model = SimpleModel(num_features)
criterion = nn.MSELoss()  # Mean Squared Error loss for regression task
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0

    for inputs_batch, targets_batch in dataloader:
        # Forward pass
        outputs = model(inputs_batch)
        loss = criterion(outputs, targets_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

In [ ]:
# Save the trained model
torch.save(model.state_dict(), 'drive/MyDrive/cs6365/baseline.pth')

# Dropping columns missing from API

In [ ]:
# Dropping unnecessary columns
train_df = df21.drop(["name", "team", "position", "fixture", "kickoff_time", "opponent_team", "round", "transfers_balance"], axis=1)

# Create a column for ground truth
ground_truth = df21.loc[:, ["element", "total_points", "GW"]]

# total points for GW 2 becomes the ground truth for GW 1
ground_truth['GW'] -= 1
ground_truth = ground_truth.rename(columns={"total_points":"y"})

# inner join with train df
train_df = train_df.merge(ground_truth, how="inner", on=["element", "GW"])

# Dropping 'element' and 'gw' columns as they are no longer needed
train_df = train_df.drop(["element", "GW"], axis=1)
train_df

In [ ]:
#Pre-processing

# Dropping columns not present in API
train_df = train_df.drop(['xP', 'selected', 'team_a_score', 'team_h_score', 'value', 'was_home'], axis=1)
train_df = train_df.astype(np.float32)
train_df

In [ ]:
train_df.columns

In [ ]:
cols = """

{'can_transact': True, 'can_select': True, 'chance_of_playing_next_round': 0, 'chance_of_playing_this_round': 0, 'code': 205651, 'cost_change_event': 0, 'cost_change_event_fall': 0, 'cost_change_start': -5, 'cost_change_start_fall': 5, 'dreamteam_count': 2, 'element_type': 4, 'ep_next': '0.0', 'ep_this': '0.0', 'event_points': 0, 'first_name': 'Gabriel', 'form': '0.0', 'id': 2, 'in_dreamteam': False, 'news': 'Knee injury - Unknown return date', 'news_added': '2025-01-12T22:00:07.802845Z', 'now_cost': 65, 'photo': '205651.jpg', 'points_per_game': '2.5', 'removed': False, 'second_name': 'Fernando de Jesus', 'selected_by_percent': '1.1', 'special': False, 'squad_number': None, 'status': 'i', 'team': 1, 'team_code': 3, 'total_points': 42, 'transfers_in': 1237850, 'transfers_in_event': 43, 'transfers_out': 1405179, 'transfers_out_event': 806, 'value_form': '0.0', 'value_season': '6.5', 'web_name': 'G.Jesus', 'region': 30, 'team_join_date': '2022-07-04', 'birth_date': '1997-04-03', 'has_temporary_code': False, 'opta_code': 'p205651', 'minutes': 600, 'goals_scored': 3, 'assists': 2, 'clean_sheets': 2, 'goals_conceded': 5, 'own_goals': 0, 'penalties_saved': 0, 'penalties_missed': 0, 'yellow_cards': 4, 'red_cards': 0, 'saves': 0, 'bonus': 6, 'bps': 152, 'influence': '154.4', 'creativity': '119.5', 'threat': '255.0', 'ict_index': '52.6', 'starts': 6, 'expected_goals': '3.05', 'expected_assists': '0.52', 'expected_goal_involvements': '3.57', 'expected_goals_conceded': '5.82', 'mng_win': 0, 'mng_draw': 0, 'mng_loss': 0, 'mng_underdog_win': 0, 'mng_underdog_draw': 0, 'mng_clean_sheets': 0, 'mng_goals_scored': 0, 'influence_rank': 306, 'influence_rank_type': 29, 'creativity_rank': 239, 'creativity_rank_type': 27, 'threat_rank': 105, 'threat_rank_type': 31, 'ict_index_rank': 232, 'ict_index_rank_type': 30, 'corners_and_indirect_freekicks_order': None, 'corners_and_indirect_freekicks_text': '', 'direct_freekicks_order': None, 'direct_freekicks_text': '', 'penalties_order': None, 'penalties_text': '', 'expected_goals_per_90': 0.46, 'saves_per_90': 0.0, 'expected_assists_per_90': 0.08, 'expected_goal_involvements_per_90': 0.54, 'expected_goals_conceded_per_90': 0.87, 'goals_conceded_per_90': 0.75, 'now_cost_rank': 42, 'now_cost_rank_type': 19, 'form_rank': 777, 'form_rank_type': 80, 'points_per_game_rank': 203, 'points_per_game_rank_type': 27, 'selected_rank': 202, 'selected_rank_type': 33, 'starts_per_90': 0.9, 'clean_sheets_per_90': 0.3}
"""

for c in train_df.columns:
  print(f"{c in cols}")

In [28]:
# Convert data to PyTorch Dataset and DataLoader
inputs = train_df.drop('y', axis=1).to_numpy()
num_features = inputs.shape[1]
print("Num features", num_features)
targets = train_df.loc[:,["y"]].to_numpy()
dataset = FplDataset(inputs, targets)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

Num features 20


In [ ]:
num_epochs = 70
lr = 5e-6
weight_decay = 1e-2

# Start a new wandb run to track this script.
run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="aamirmd-georgia-institute-of-technology",
    # Set the wandb project where this run will be logged.
    project="cs6365-fpl-model",
    # Track hyperparameters and run metadata.
    config={
        "learning_rate": lr,
        "architecture": "ResidualModel",
        "epochs": num_epochs,
        "weight_decay": weight_decay
    },
)

In [33]:
class ResidualBlock(nn.Module):
    def __init__(self, dim, dropout=0.1):
        super().__init__()
        self.fc1 = nn.Linear(dim, dim)
        self.fc2 = nn.Linear(dim, dim)
        self.norm1 = nn.LayerNorm(dim)
        self.norm2 = nn.LayerNorm(dim)
        self.dropout = nn.Dropout(dropout)
        self.activation = nn.LeakyReLU(0.1)

    def forward(self, x):
        residual = x
        out = self.fc1(self.norm1(x))
        out = self.activation(out)
        out = self.dropout(out)
        out = self.fc2(self.norm2(out))
        return self.activation(out + residual)

# class RegressionModel(nn.Module):
#     def __init__(self, input_dim, output_dim=1, hidden_dim=128, num_blocks=3, dropout=0.1):
#         super().__init__()
#         self.input_layer = nn.Linear(input_dim, hidden_dim)
#         self.blocks = nn.Sequential(*[
#             ResidualBlock(hidden_dim, dropout) for _ in range(num_blocks)
#         ])
#         self.output_layer = nn.Linear(hidden_dim, output_dim)

#     def forward(self, x):
#         x = self.input_layer(x)
#         x = self.blocks(x)
#         return self.output_layer(x)

# Define a more complex
class FplModel(nn.Module):
    def __init__(self, num_features, output_dim=1, hidden_dim=128, num_blocks=3, dropout=0.1):
        super(FplModel, self).__init__()
        self.input_layer = nn.Linear(num_features, hidden_dim)
        self.blocks = nn.Sequential(*[
            ResidualBlock(hidden_dim, dropout) for _ in range(num_blocks)
        ])
        self.output_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.input_layer(x)
        x = self.blocks(x)
        return self.output_layer(x)

# Initialize the model, loss function, and optimizer
model = FplModel(num_features)
criterion = nn.MSELoss()  # Mean Squared Error loss for regression task
optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

In [34]:
# Training loop
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0

    for inputs_batch, targets_batch in dataloader:
        # Forward pass
        outputs = model(inputs_batch)
        loss = criterion(outputs, targets_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(dataloader)
    run.log({'avg_loss': avg_loss})
    if (epoch+1) % 5 == 0:
      print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

run.finish()

Epoch [5/70], Loss: 220184.0929
Epoch [10/70], Loss: 13316.0449
Epoch [15/70], Loss: 2410.3823
Epoch [20/70], Loss: 709.8140
Epoch [25/70], Loss: 271.9349
Epoch [30/70], Loss: 154.9150
Epoch [35/70], Loss: 67.6448
Epoch [40/70], Loss: 107.0118
Epoch [45/70], Loss: 41.0262
Epoch [50/70], Loss: 41.3879
Epoch [55/70], Loss: 42.6689
Epoch [60/70], Loss: 31.9948
Epoch [65/70], Loss: 43.9172
Epoch [70/70], Loss: 48.7768


avg_loss,█▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
avg_loss,48.77683


In [35]:
# Save the trained model
torch.save(model.state_dict(), 'drive/MyDrive/cs6365/residual-1.pth')